In [ ]:
import pandas as pd

In [ ]:
file = r"..\metadata_20241010T1031.csv"

river_name_list = ["Nelson River"]

In [ ]:
df = pd.read_csv(file, encoding="utf-8-sig")
df.head()

In [ ]:
def modify_river_name(river):
    river_dict = {
        "Mackenzie River": "Mackenzie",
        "Coppermine River": "Coppermine",
        "Big River": "Big",
        "Tree River": "Tree",
        "Burnside River": "Burnside",
        "Gordon River": "Gordon",
        "Ellice River": "Ellice",
        "Back River": "Back",
        "Kazan River": "Kazan",
    }
    return river_dict.get(river, river)


def modify_station_name(station):
    station_dict = {
        "7.5 Km Down Of Buur'S Mouth": "7.5 km down of Buur's mouth",
    }
    return station_dict.get(station, station)


def extract_river_station(river_station):
    keywords = [" Near ", " Nr ", "Above", "Below", " River ", " - "]
    keywords2 = [" Near ", " Nr ", "Above", "Below"]
    river_station = river_station.title()

    for river_name in river_name_list:
        if river_station.startswith(river_name):
            # rest of the string is station name
            return river_name, river_station.replace(river_name, "").strip()

    if " At " in river_station:
        river_station = river_station.split(" At ", maxsplit=1)
        assert len(river_station) == 2
        river = river_station[0]
        station = river_station[1]
        return river.strip(), station.strip()

    for keyword in keywords:
        if keyword in river_station:
            river_station = river_station.split(keyword, maxsplit=1)
            assert len(river_station) == 2
            river = river_station[0]
            station = river_station[1]
            if keyword in keywords2:
                station = keyword + station
            return river.strip(), station.strip()

    return river_station, ""

In [ ]:
df[["River", "Station"]] = df.apply(
    lambda x: extract_river_station(x["Station Name"]), axis=1, result_type="expand"
)

df["River"] = df["River"].apply(modify_river_name)
df["Station"] = df["Station"].apply(modify_station_name)

df.rename(columns={"Latitude": "Lat", "Longitude": "Lon"}, inplace=True)

In [ ]:
df

In [ ]:
df.to_csv(r"..\Station Info.csv", index=False, encoding="utf-8-sig")